In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

def clean_employee_records(input_file, output_file, handle_missing_id='generate'):

    # Read the CSV file
    df = pd.read_csv(input_file)

    # 1. Remove duplicate rows (keeping first occurrence)
    df_clean = df.drop_duplicates()

    # 2. Handle missing EmployeeIDs based on user preference
    missing_id_mask = df_clean['EmployeeID'].isna() | (df_clean['EmployeeID'] == '')

    if handle_missing_id == 'drop':
        df_clean = df_clean[~missing_id_mask]
    elif handle_missing_id == 'generate':
        # Find the highest existing EmployeeID number
        existing_ids = df_clean['EmployeeID'].dropna()
        numeric_parts = [int(id[1:]) for id in existing_ids if id != '' and id[1:].isdigit()]
        next_id_num = max(numeric_parts) + 1 if numeric_parts else 1

        # Generate new IDs for rows with missing EmployeeID
        for idx in df_clean[missing_id_mask].index:
            df_clean.at[idx, 'EmployeeID'] = f"E{next_id_num:03d}"
            next_id_num += 1
    elif handle_missing_id == 'keep':
        pass

    # 3. Clean Salary - handle missing values and ensure numeric format
    df_clean['Salary($)'] = df_clean['Salary($)'].replace('', np.nan)
    df_clean['Salary($)'] = pd.to_numeric(df_clean['Salary($)'], errors='coerce')

    # Calculate mean salary (excluding missing values)
    mean_salary = df_clean['Salary($)'].mean()

    # Fill missing salaries with mean salary
    df_clean['Salary($)'] = df_clean['Salary($)'].fillna(mean_salary)

    # 4. Clean Department - fill missing values with 'Unknown'
    df_clean['Department'] = df_clean['Department'].fillna('Unknown')
    df_clean['Department'] = df_clean['Department'].replace('', 'Unknown')

    # 5. Clean JoinDate - standardize date format and handle missing values
    def parse_date(date_str):
        if pd.isna(date_str) or date_str == '':
            return np.nan

        formats = ['%Y-%m-%d', '%Y/%m/%d', '%d-%m-%Y', '%d/%m/%Y']

        for fmt in formats:
            try:
                return datetime.strptime(str(date_str), fmt)
            except ValueError:
                continue

        return np.nan

    # Parse dates first (returns datetime objects)
    df_clean['JoinDate_parsed'] = df_clean['JoinDate'].apply(parse_date)

    # Calculate mean date (excluding missing values)
    date_timestamps = df_clean['JoinDate_parsed'].dropna().apply(lambda x: x.timestamp())
    mean_timestamp = date_timestamps.mean()

    if not pd.isna(mean_timestamp):
        mean_date = datetime.fromtimestamp(mean_timestamp)
        # Fill missing dates with mean date
        df_clean['JoinDate_parsed'] = df_clean['JoinDate_parsed'].fillna(mean_date)

    # Convert back to string format for final output
    df_clean['JoinDate'] = df_clean['JoinDate_parsed'].apply(
        lambda x: x.strftime('%Y-%m-%d') if pd.notna(x) else np.nan
    )

    # Drop the temporary parsed column
    df_clean = df_clean.drop('JoinDate_parsed', axis=1)

    # 6. Reset index after cleaning
    df_clean = df_clean.reset_index(drop=True)

    # 7. Save cleaned data to new CSV file
    df_clean.to_csv(output_file, index=False)

    return df_clean

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Usage example:
if __name__ == "__main__":
    cleaned_df = clean_employee_records('employee_records_dirty.csv', 'employee_records_clean.csv', 'generate')